In [1]:
import os
from datetime import datetime, timezone, timedelta

import pandas as pd
import pytz
from dotenv import load_dotenv

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetOrdersRequest, GetPortfolioHistoryRequest
from alpaca.trading.enums import QueryOrderStatus, OrderSide, OrderStatus
from alpaca.common.exceptions import APIError

In [2]:
class AccountManager:
    """
    Manages account and portfolio tasks (excluding order placement).
    """

    def __init__(self, paper=True):
        """Initialize the Alpaca Trading Client for account management."""
        # Load environment variables
        load_dotenv()

        self.API_KEY = os.getenv("ALPACA_API_KEY")
        self.SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")

        if not self.API_KEY or not self.SECRET_KEY:
            raise ValueError("Missing API credentials. Ensure they are set in the .env file.")

        # Initialize Alpaca Trading Client (paper trading mode enabled by default)
        self.client = TradingClient(self.API_KEY, self.SECRET_KEY, paper=paper)

    def get_closed_positions(self, symbols=None, after_time_ct=None, until_time_ct=None):
        """
        Fetch closed positions from trade history between a specified time range (Central Time).
        Converts 'after' and 'until' timestamps from CT to UTC before sending the request.
        """
        try:
            ct_tz = pytz.timezone("America/Chicago")  # Central Time zone
            utc_tz = pytz.utc  # UTC Time zone

            # Convert 'after' time from CT to UTC
            if after_time_ct:
                after_dt_ct = pd.to_datetime(after_time_ct).tz_localize(ct_tz)
                after_dt_utc = after_dt_ct.astimezone(utc_tz)
                print(f"Fetching closed positions after {after_dt_utc} UTC ({after_dt_ct} CT)")
            else:
                after_dt_utc = datetime.now(pytz.timezone("America/New_York")).date() - timedelta(days=1)  # Default: yesterday
            
            # Convert 'until' time from CT to UTC (if provided)
            until_dt_utc = None
            if until_time_ct:
                until_dt_ct = pd.to_datetime(until_time_ct).tz_localize(ct_tz)
                until_dt_utc = until_dt_ct.astimezone(utc_tz)
                print(f"Fetching closed positions until {until_dt_utc} UTC ({until_dt_ct} CT)")

            # Build API request with 'after' and 'until' filters
            request = GetOrdersRequest(
                status=QueryOrderStatus.CLOSED,
                after=after_dt_utc,
                until=until_dt_utc,  # Add until filter
                limit=500,
                symbols=symbols if symbols else None
            )

            orders = self.client.get_orders(request)

            # Convert objects to dictionaries
            orders_dicts = [order.model_dump() if hasattr(order, 'dict') else order.__dict__ for order in orders]

            # Create DataFrame
            df = pd.DataFrame(orders_dicts)

            # Define the columns of interest
            columns_of_interest = [
                'id', 'filled_at', 'symbol', 'qty', 'filled_qty', 
                'filled_avg_price', 'order_type', 'side', 'status', 'stop_price', 'trail_price'
            ]

            # Filter DataFrame
            df = df[columns_of_interest]

            # Convert necessary columns to correct data types
            df['filled_avg_price'] = pd.to_numeric(df['filled_avg_price'], errors='coerce')
            df['filled_qty'] = pd.to_numeric(df['filled_qty'], errors='coerce')
            df['stop_price'] = pd.to_numeric(df['stop_price'], errors='coerce')
            df['trail_price'] = pd.to_numeric(df['trail_price'], errors='coerce')

            # Convert 'filled_at' to datetime
            df['filled_at'] = pd.to_datetime(df['filled_at'], errors='coerce')

            # Check if 'filled_at' is timezone-aware
            if df['filled_at'].dt.tz is None:
                # If it's naïve, first localize to UTC before converting
                df['filled_at'] = df['filled_at'].dt.tz_localize('UTC')

            # Convert to Central Time (CT)
            df['filled_at'] = df['filled_at'].dt.tz_convert('America/Chicago')

            # Drop any rows where 'filled_at' is still missing after conversion
            df = df.dropna(subset=['filled_at'])

            return df

        except APIError as e:
            print(f"❌ Error fetching closed positions: {e}")
            return []


In [3]:
api = AccountManager()
df = api.get_closed_positions(
    after_time_ct="2025-03-13 08:40 AM",
    until_time_ct="2025-03-13 05:00 PM"
)

print(df.shape)
print(df.symbol.value_counts())
df.tail(5)

Fetching closed positions after 2025-03-13 13:40:00+00:00 UTC (2025-03-13 08:40:00-05:00 CT)
Fetching closed positions until 2025-03-13 22:00:00+00:00 UTC (2025-03-13 17:00:00-05:00 CT)
(64, 11)
symbol
SPY      10
QQQ       8
AAPL      8
XLE       6
MSFT      6
GOOGL     6
AMZN      6
IWM       6
META      4
VXX       2
NVDA      2
Name: count, dtype: int64


,id,filled_at,symbol,qty,filled_qty,filled_avg_price,order_type,side,status,stop_price,trail_price
67,553eb227-58c0-4eb0-99a0-17c479291866,2025-03-13 09:30:05.935480-05:00,AAPL,17,17,214.369412,OrderType.MARKET,OrderSide.SELL,OrderStatus.FILLED,NaN,NaN
68,97b3ea6b-ed23-4cf5-9184-e7feb42304e0,2025-03-13 09:30:04.173950-05:00,XLE,12,12,88.280000,OrderType.MARKET,OrderSide.BUY,OrderStatus.FILLED,NaN,NaN
69,b3912dbd-380d-47bb-b186-4bd1d1e35c43,2025-03-13 09:30:03.420139-05:00,IWM,19,19,200.671053,OrderType.MARKET,OrderSide.SELL,OrderStatus.FILLED,NaN,NaN
70,8009e4f9-29c5-4e97-93f1-96a0810a11fb,2025-03-13 09:30:01.995324-05:00,QQQ,8,8,473.962500,OrderType.MARKET,OrderSide.SELL,OrderStatus.FILLED,NaN,NaN
71,c71960e3-0041-49c3-bab0-cc179eb41db2,2025-03-13 09:30:00.415280-05:00,SPY,7,7,557.230000,OrderType.MARKET,OrderSide.SELL,OrderStatus.FILLED,NaN,NaN


In [4]:
# Assume df is your DataFrame with all orders already converted (including 'filled_at' in CT)

pnl_records = []

# Group orders by symbol
for symbol, group in df.groupby('symbol'):
    # Sort the orders for this symbol by filled_at
    group = group.sort_values(by='filled_at').reset_index(drop=True)
    i = 0
    while i < len(group) - 1:
        order1 = group.iloc[i]
        order2 = group.iloc[i+1]
        
        # For pairing, ensure the filled_qty match
        if order1['filled_qty'] != order2['filled_qty']:
            i += 1
            continue
        
        # Use substring matching for order types & sides
        is_market_order1 = "MARKET" in str(order1['order_type'])
        is_market_order2 = "MARKET" in str(order2['order_type'])
        is_trailing_stop_order2 = "TRAILING_STOP" in str(order2['order_type'])
        
        # Convert timestamps (they're already in CT)
        market_filled_at = pd.to_datetime(order1['filled_at'])
        second_filled_at = pd.to_datetime(order2['filled_at'])
        
        # Case 1: Market order followed by a Trailing Stop order
        if is_market_order1 and is_trailing_stop_order2:
            if (("SELL" in str(order1['side']) and "BUY" in str(order2['side'])) or
                ("BUY" in str(order1['side']) and "SELL" in str(order2['side']))):
                qty = order1['filled_qty']
                mkt_price = order1['filled_avg_price']
                trailing_price = order2['filled_avg_price']
                
                # Short trade: (Sell - Buy) * Qty | Long trade: (Buy - Sell) * Qty
                pnl = (mkt_price - trailing_price) * qty if "SELL" in str(order1['side']) else (trailing_price - mkt_price) * qty
                
                pnl_records.append({
                    'symbol': symbol,
                    'market_order_filled_at_ct': market_filled_at,
                    'trailing_stop_filled_at_ct': second_filled_at,
                    'market_order_type': order1['order_type'],
                    'market_order_side': order1['side'],
                    'market_order_price': mkt_price,
                    'trailing_stop_order_price': trailing_price,
                    'qty': qty,
                    'pnl': pnl
                })
                i += 2
                continue

        # Case 2: End-of-Day closing position: Market order followed by Market order (BUY → SELL or SELL → BUY)
        if is_market_order1 and is_market_order2:
            if (("SELL" in str(order1['side']) and "BUY" in str(order2['side'])) or
                ("BUY" in str(order1['side']) and "SELL" in str(order2['side']))):
                
                qty = order1['filled_qty']
                first_price = order1['filled_avg_price']
                second_price = order2['filled_avg_price']
                
                # Short trade (Sell → Buy) | Long trade (Buy → Sell)
                pnl = (first_price - second_price) * qty if "SELL" in str(order1['side']) else (second_price - first_price) * qty
                
                pnl_records.append({
                    'symbol': symbol,
                    'first_order_filled_at_ct': market_filled_at,
                    'second_order_filled_at_ct': second_filled_at,
                    'first_order_side': order1['side'],
                    'second_order_side': order2['side'],
                    'first_price': first_price,
                    'second_price': second_price,
                    'qty': qty,
                    'pnl': pnl
                })
                i += 2
                continue

        # If no pair found, move to the next order
        i += 1

# Convert PnL records to DataFrame
df_pnl = pd.DataFrame(pnl_records)

# Print total realized PnL across all securities
print(f'Total PnL: {round(df_pnl.pnl.sum(), 2)}')

# Compute PnL grouped by stock symbol
pnl_by_stock = df_pnl.groupby('symbol', as_index=False)['pnl'].sum().sort_values(by='pnl', ascending=False)
display(pnl_by_stock)

Total PnL: 205.71


,symbol,pnl
1,AMZN,53.489997
0,AAPL,33.540014
3,IWM,25.300001
2,GOOGL,24.900018
7,QQQ,24.490000
4,META,22.260000
8,SPY,16.690005
9,VXX,11.319990
5,MSFT,6.200001
10,XLE,-3.779996
